### 1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.
### 2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля)

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import re
import pandas as pd
import json
from pymongo import MongoClient
from selenium import webdriver
import time
from pprint import pprint

In [2]:
client = MongoClient('localhost', 27017)
db = client['pacring_job_sites_db']
collection = db['hh_superjobs']

In [3]:
# Определяем параметры

vac = input("Какую должность ищем: ")
max_list_sj = int(input("Сколько страниц Superjobs анализировать: "))
max_list_hh = int(input("Сколько страниц HH анализировать: "))

Какую должность ищем: Python
Сколько страниц Superjobs анализировать: 1
Сколько страниц HH анализировать: 1


In [4]:
#Идея привести все ЗП к одному виду мне понравилась, попробовал тоже реализовать это.
cur_req = requests.get("https://www.cbr-xml-daily.ru/daily_json.js")    
cur = json.loads(cur_req.text)  

In [5]:
# Забираем инфу с HH.ru



for x in range(max_list_hh):
    #Т.к. hh не отдает все карточки а только 20, пришлось использовать silenium чтобы сначала расскрывать страничку, потом парсить.
    driver = webdriver.Chrome(executable_path=r"..\chromedriver.exe")
    driver.get(f'https://hh.ru/search/vacancy?area=1&fromSearchLine=true&text={vac}&from=suggest_post&page={x}')
    time.sleep(5)
    html = driver.page_source
    #resphh = requests.get(f'https://hh.ru/search/vacancy?area=1&fromSearchLine=true&text={vac}&from=suggest_post&page={x}', headers = {'User-agent':'Chrome/63.0.3239.108'}).text
    soup = bs(html, 'html.parser')
    cards = soup.find_all(name='div', attrs={"class" : "vacancy-serp-item"} )
    for i in cards:
        name_job = i.find('a', attrs = {'data-qa': "vacancy-serp__vacancy-title"}).text
        
        link_job = i.find('a', attrs = {'data-qa': "vacancy-serp__vacancy-title"}).get('href')
        
        employer = i.find('a', attrs = {'data-qa': "vacancy-serp__vacancy-employer"}).text
        employer_link = f"https://hh.ru{i.find('a', attrs = {'data-qa': 'vacancy-serp__vacancy-employer'}).get('href')}"
        
        address = i.find('div', attrs = {'data-qa': "vacancy-serp__vacancy-address"}).text
        
        cost = i.find('span', attrs = {'data-qa' : "vacancy-serp__vacancy-compensation"})
        min_cost = np.nan
        max_cost = np.nan
        if cost:
            cost = cost.text 
            cost = cost.replace("\u202f", "").replace(" ", "")
            if re.findall('^от[0-9]+', cost):
                min_cost = int(re.findall('[0-9]+', cost)[0])
            elif re.findall('^до[0-9]+', cost):
                max_cost = int(re.findall('[0-9]+', cost)[0])
            else:
                min_cost = int(re.findall('[0-9]+', cost)[0])
                max_cost = int(re.findall('[0-9]+', cost)[1])
            if not re.findall('руб.', cost):
                currency = re.findall('[A-Z]{2,5}', cost)[0]
                if not np.isnan(min_cost):
                    min_cost = int(min_cost*cur["Valute"][currency]["Value"])//1000*1000
                if not np.isnan(max_cost):
                    max_cost = int(max_cost*cur["Valute"][currency]["Value"])//1000*1000
        #Собираем серию
        
        job_info = {}
        job_info['Наименование должности'] = name_job
        job_info['Ссылка на позицию'] = link_job
        job_info['Минимальная ЗП'] = min_cost
        job_info['Максимальная ЗП'] = max_cost
        job_info['Сайт сбора позиции'] = "hh.ru"
        job_info['Работадатель'] = employer
        job_info['Ссылка на работодателя'] = employer_link
        job_info['Место работы'] = address
        
        
        #Вставляем в БД
        
        if not collection.find_one({'Ссылка на позицию' : job_info['Ссылка на позицию']}):
            collection.insert_one(job_info)        


<ipython-input-5-6f90987c7f41>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"..\chromedriver.exe")


In [6]:
# Забираем инфу с Superjobs.ru



for x in range(max_list_sj):
    resphh = requests.get(f'https://www.superjob.ru/vacancy/search/?keywords={vac}&page={x}', headers = {'User-agent':'Mozilla/5.0'}).text
    soup = bs(resphh, 'html.parser')
    cards = soup.find_all(name='div', attrs={"class" : "f-test-vacancy-item"})
    
    for i in cards:
        name_job = i.find('a', attrs = {'class': "_6AfZ9"}).text
        
        
        link_job = f"https://www.superjob.ru{i.find('a', attrs = {'class': '_6AfZ9'}).get('href')}"
        employer = i.find('a', attrs = {'class': "_205Zx"})
        if employer:
            employer = i.find('a', attrs = {'class': "_205Zx"}).text
        
            employer_link = f"https://www.superjob.ru{i.find('a', attrs = {'class': '_205Zx'}).get('href')}"
        else:
            employer = np.nan
            employer_link = np.nan

        address = i.find('span', attrs = {'class': 'f-test-text-company-item-location'}).find(lambda tag: len(tag.attrs) == 0).text
             
        cost = i.find('span', attrs = {'class' : '_2Wp8I'})
         
        cost = cost.text
        cost = cost.replace("\xa0", "").replace(" ", "")
        if not re.findall('[0-9]+', cost):
            min_cost = np.nan
            max_cost = np.nan                   
        elif re.findall('^от[0-9]+', cost):
            min_cost = int(re.findall('[0-9]+', cost)[0])
        elif re.findall('^до[0-9]+', cost):
            max_cost = int(re.findall('[0-9]+', cost)[0])
        elif len(re.findall('[0-9]+', cost)) == 1:
            min_cost = int(re.findall('[0-9]+', cost)[0])
            max_cost = int(re.findall('[0-9]+', cost)[0])

        else:
            min_cost = int(re.findall('[0-9]+', cost)[0])
            max_cost = int(re.findall('[0-9]+', cost)[1])
        
        if not re.findall('руб.', cost) and re.findall('[0-9]+', cost):
            currency = re.findall('[A-Z]{2,5}', cost)[0]
            if not np.isnan(min_cost):
                min_cost = int(min_cost*cur["Valute"][currency]["Value"])//1000*1000
            if not np.isnan(max_cost):
                max_cost = int(max_cost*cur["Valute"][currency]["Value"])//1000*1000
                
        job_info = {}
        job_info['Наименование должности'] = name_job
        job_info['Ссылка на позицию'] = link_job
        job_info['Минимальная ЗП'] = min_cost
        job_info['Максимальная ЗП'] = max_cost
        job_info['Сайт сбора позиции'] = "superjob.ru"
        job_info['Работадатель'] = employer
        job_info['Ссылка на работодателя'] = employer_link
        job_info['Место работы'] = address
        
        #Вставляем в БД
        
        if not collection.find_one({'Ссылка на позицию' : job_info['Ссылка на позицию']}):
            collection.insert_one(job_info)
        
        #
        
           


In [7]:
my_salary = int(input("От какой ЗП выводить: "))

От какой ЗП выводить: 9999999


In [8]:
for i in collection.find({ "$or" : [{'Минимальная ЗП':{"$gt": my_salary}}, {'Максимальная ЗП':{"$gt": my_salary}}]}):
    pprint(i)